In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import re

In [2]:
import os.path
from os import path

In [3]:
# Retorna os resultados (tabelas) do site Fundamentus para dado "papel", requer como input o Ticker completo do ativo.
def getResults(papel: str):
    r = requests.get("https://www.fundamentus.com.br/detalhes.php?papel="+ papel)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all('table', attrs={'class': 'w728'})
    return results

In [4]:
# Transforma String em tipo Float, ex: 150.000,90 -> 150000.90
def transformFloat(val: str):
    try:
        return float(val.replace(".", "").replace(",", "."))
    except ValueError:
        return "-"

In [5]:
# Transforma String com Percentual em tipo Float, ex: %1.000,90 -> 10.0009
def transformPercent(val: str):
    try:
        return round(float(val.strip('%').replace(".", "").replace(",", "."))/100, 5)
    except ValueError:
        return "-"

In [6]:
# Transforma String removendo "?"
def transformKey(val: str):
    try:
        return val.strip('?')
    except ValueError:
        return "-"

In [7]:
# Transforma String removendo espaço e novas linhas
def transformText(val: str):
    try:
        return val.replace("\n", "").strip()
    except ValueError:
        return "-"

In [8]:
# Handler para as strings
def handleStr(val: str):
    if("%" in val):
        return transformPercent(val)
    elif(re.search('[a-zA-Z]', val) or "/" in val):
        return transformText(val)
    else:
        return transformFloat(val)

In [9]:
# Identificador de Oscilações para formatação no excel
def isOscilacao(val: str):
    return val in ["Dia", "Mês", "30 dias", "12 meses", "2020", "2019", "2018", "2017", "2016", "2015"]

In [28]:
# Verificando se arquivo base.xlsx existe e não está vazio
if not path.exists("base.xlsx") or len(pd.read_excel("base.xlsx", header=None)) == 0:
    print("Arquivo 'base.xlsx' não foi encontrado ou está vazio.")
else:
    # leitura do arquivo chamado base.xlsx (deve ser criado previamente) 
    # que contem a lista de Tickers a serem buscados na primeira coluna.
    base_df = pd.read_excel("base.xlsx", header=None)

    ### Setup inicial ###
    _dict1 = {}
    _dict4 = {}
    _dict5 = {}
    _dict5_3m = {}
    _dict = {}

    # Cria pasta chamada acoes caso não exista na pasta local
    if not os.path.exists("acoes"):
        os.makedirs("acoes")

    # Para cada ticker, busca os dados na Fundamentus e cria ou atualiza arquivo com as informações mais recentes
    for papel in base_df[0]:
        filename = papel + "_fun.xlsx"
        filepath = 'acoes/' + filename
        results = getResults(papel)
        if(len(results) != 5):
            print("Não foi possível obter dados para o papel: "+ papel)
        else:
            ### Criando DataFrame com os dados do Fundamentus. ###
            # Primeira Tabela
            r1 = results[0].find_all('td')
            i = 1

            tab = r1

            papel = tab[i].text.replace("\n", "").strip()
            cotacao = transformFloat(tab[i + 2].text.replace("\n", "").strip())
            tipo = tab[i + 4].text.replace("\n", "").strip()
            data = tab[i + 6].text.replace("\n", "").strip()
            empresa = tab[i + 8].text.replace("\n", "").strip()
            min_52 = transformFloat(tab[i + 10].text.replace("\n", "").strip())
            setor = tab[i + 12].text.replace("\n", "").strip()
            max_52 = transformFloat(tab[i + 14].text.replace("\n", "").strip())
            subsetor = tab[i + 16].text.replace("\n", "").strip()
            vol_med = transformFloat(tab[i + 18].text.replace("\n", "").strip())

            # Segunda Tabela
            r2 = results[1].find_all('td')
            i = 1

            tab = r2

            valor_mercado = transformFloat(tab[i].text.replace("\n", "").strip())
            ult_balanco = tab[i + 2].text.replace("\n", "").strip()
            valor_firma = transformFloat(tab[i + 4].text.replace("\n", "").strip())
            nro_acoes = transformFloat(tab[i + 6].text.replace("\n", "").strip())

            # Terceira Tabela (Oscilações & Indicadores fundamentalistas)
            r3 = results[2].find_all('td')
            i = 1

            tab = r3

            osc_dia = transformPercent(tab[i + 2].text.replace("\n", "").strip())
            P_L = transformFloat(tab[i + 4].text.replace("\n", "").strip())
            LPA = transformFloat(tab[i + 6].text.replace("\n", "").strip())
            osc_mes = transformPercent(tab[i + 8].text.replace("\n", "").strip())
            P_VP = transformFloat(tab[i + 10].text.replace("\n", "").strip())
            VPA = transformFloat(tab[i + 12].text.replace("\n", "").strip())
            osc_30d = transformPercent(tab[i + 14].text.replace("\n", "").strip())
            P_EBIT = transformFloat(tab[i + 16].text.replace("\n", "").strip())
            marg_bruta = transformPercent(tab[i + 18].text.replace("\n", "").strip())
            osc_12m = transformPercent(tab[i + 20].text.replace("\n", "").strip())
            PSR = transformFloat(tab[i + 22].text.replace("\n", "").strip())
            marg_EBIT = transformPercent(tab[i + 24].text.replace("\n", "").strip())
            osc_2020 = transformPercent(tab[i + 26].text.replace("\n", "").strip())
            P_Ativos = transformFloat(tab[i + 28].text.replace("\n", "").strip())
            marg_liq = transformPercent(tab[i + 30].text.replace("\n", "").strip())
            osc_2019 = transformPercent(tab[i + 32].text.replace("\n", "").strip())
            P_CapGiro = transformFloat(tab[i + 34].text.replace("\n", "").strip())
            EBIT_Ativo = transformPercent(tab[i + 36].text.replace("\n", "").strip())
            osc_2018 = transformPercent(tab[i + 38].text.replace("\n", "").strip())
            P_AtivCircLiq = transformFloat(tab[i + 40].text.replace("\n", "").strip())
            ROIC = transformPercent(tab[i + 42].text.replace("\n", "").strip())
            osc_2017 = transformPercent(tab[i + 44].text.replace("\n", "").strip())
            div_yield = transformPercent(tab[i + 46].text.replace("\n", "").strip())
            ROE = transformPercent(tab[i + 48].text.replace("\n", "").strip())
            osc_2016 = transformPercent(tab[i + 50].text.replace("\n", "").strip())
            EV_EBITDA = transformFloat(tab[i + 52].text.replace("\n", "").strip())
            liq_corrente = transformFloat(tab[i + 54].text.replace("\n", "").strip())
            osc_2015 = transformPercent(tab[i + 56].text.replace("\n", "").strip())
            EV_EBIT = transformFloat(tab[i + 58].text.replace("\n", "").strip())
            DivBruta_Patrim = transformFloat(tab[i + 60].text.replace("\n", "").strip())
            cresc_rec_5a = transformPercent(tab[i + 64].text.replace("\n", "").strip())
            giro_ativos = transformFloat(tab[i + 66].text.replace("\n", "").strip())

            # Quarta Tabela (Dados Balanço Patrimonial)
            r4 = results[3].find_all('td')

            for index, obj in enumerate(r4):
                if "?" in obj.text:
                    _dict4[transformKey(obj.text)] = handleStr(r4[index + 1].text)

            # Quinta Tabela (Dados demonstrativos de resultados)
            r5 = results[4].find_all('td')

            for index, obj in enumerate(r5):
                if "?" in obj.text:
                    if "12_meses_" + transformKey(obj.text) in _dict5.keys():
                        _dict5_3m["3_meses_" + transformKey(obj.text)] = handleStr(r5[index + 1].text)
                    else: 
                        _dict5["12_meses_" + transformKey(obj.text)] = handleStr(r5[index + 1].text)

            _dict1 = {
                'papel': papel,
                'tipo': tipo,
                'empresa': empresa,
                'setor': setor,
                'subsetor': subsetor, 
                'data': data,
                'cotacao': cotacao,
                'min.52': min_52,
                'max.52': max_52,
                'vol.med': vol_med,
                'ult.balanco': ult_balanco,
                'valor.mercado': valor_mercado,
                'valor.firma': valor_firma,
                'nro.acoes': nro_acoes,
                'osc.dia': osc_dia,
                'osc.mes' : osc_mes,
                'osc.30d' : osc_30d,
                'osc.12m' : osc_12m,
                'osc.2020' : osc_2020,
                'osc.2019' : osc_2019,
                'osc.2018' : osc_2018,
                'osc.2017' : osc_2017,
                'osc.2016' : osc_2016,
                'osc.2015' : osc_2015,
                'P/L' : P_L,
                'P/VP' : P_VP,
                'P/EBIT' : P_EBIT,
                'PSR' : PSR,
                'P/Ativos' : P_Ativos,
                'P/CapGiro' : P_CapGiro,
                'P/AtivCircLiq' : P_AtivCircLiq,
                'div.yield' : div_yield,
                'EV/EBITDA' : EV_EBITDA,
                'EV/EBIT' : EV_EBIT,
                'cresc.rec.5a' : cresc_rec_5a,
                'LPA' : LPA,
                'VPA' : VPA,
                'marg.bruta' : marg_bruta,
                'marg.EBIT' : marg_EBIT,
                'marg.liq' : marg_liq,
                'EBIT/Ativo' : EBIT_Ativo,
                'ROIC' : ROIC,
                'ROE' : ROE,
                'liq.corrente' : liq_corrente,
                'DivBruta/Patrim' : DivBruta_Patrim,
                'giro.ativos' : giro_ativos
            }

            _dict = {**_dict1, **_dict4, **_dict5, **_dict5_3m}

            today = data
            keys = pd.DataFrame(_dict.keys(), columns=['Dados'])
            s = pd.DataFrame(_dict.items(), columns=['Date', today])

            if(path.exists(filepath)):
                print(filename + " já existe, anexando dados do dia.")
            else:
                print("Criando novo arquivo: " + filename)
                writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
                keys.to_excel(writer, sheet_name=papel, startrow=0, index=False)
                writer.close()

            papel_df = pd.read_excel(filepath)

            if(today in papel_df.columns):
                print("Dados do dia já estão no arquivo: " + filename)
            else:
                print("Anexando Fundamentus do dia.")
                ### Anexando Informações do dia. ###
                writer = pd.ExcelWriter(filepath, engine='xlsxwriter')

                df = pd.read_excel(filepath)

                df.insert(1, today, s[today])
                df.to_excel(writer, sheet_name=papel, index=False)

                workbook  = writer.book
                worksheet = writer.sheets[papel]

                # Formato do Header
                header_format = workbook.add_format({
                    'bold': True,
                    'valign': 'center',
                    'fg_color': '#D7E4BC',
                    'num_format': 'dd/mm/yyyy',
                    'border': 1})

                for col_num, value in enumerate(df.columns.values):
                    worksheet.write(0, col_num, value, header_format)

                # Formato das linhas
                number_format = workbook.add_format({'num_format': '#,##0;[Red]-#,##0'})
                number_format_decimal = workbook.add_format({'num_format': '#,##0.00;[Red]-#,##0.00'})
                percent_format = workbook.add_format({'num_format': '0.00%;[Red]-0.00%'})

                formato_texto = [1,2,3,4,5]
                formato_data = [6,11]
                formato_numbers = [10,12,13,14,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66]
                formato_numbers_decimal = [7,8,9,25,26,27,28,29,30,31,33,34,36,37,44,45,46]
                formato_percent = [15,16,17,18,19,20,21,22,23,24,32,35,38,39,40,41,42,43]

                for i in formato_numbers:
                    worksheet.set_row(i, None, number_format)

                for i in formato_numbers_decimal:
                    worksheet.set_row(i, None, number_format_decimal)

                for i in formato_percent:
                    worksheet.set_row(i, None, percent_format)

                # Formato das colunas
                worksheet.set_column(0, 0, 15) 
                worksheet.set_column(1, 1000, 16)

                # Fecha o Excel writer e salva o arquivo.
                writer.save()
            print("Papel: " + papel + " finalizado.")

Criando novo arquivo: PETR4_fun.xlsx
Anexando Fundamentus do dia.
Papel: PETR4 finalizado.
Criando novo arquivo: ITUB4_fun.xlsx
Anexando Fundamentus do dia.
Papel: ITUB4 finalizado.
Criando novo arquivo: VALE3_fun.xlsx
Anexando Fundamentus do dia.
Papel: VALE3 finalizado.


### Consolidando todas ações em um só arquivo. (Formatação é perdida)

In [29]:
writer = pd.ExcelWriter('Consolidado.xlsx')
for file in os.listdir("acoes"):
    if file.endswith(".xlsx"):
        print(file)
        df = pd.read_excel("acoes/" + file)
        df.to_excel(writer, df[df['Dados'] == 'papel'].values[0][1], index=None)
        continue
    else:
        continue
writer.save()

ITUB4_fun.xlsx
PETR4_fun.xlsx
VALE3_fun.xlsx
